In [1]:
# 웹크롤링용 라이브러리
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

# 기타 라이브러리
import time
import warnings
warnings.filterwarnings('ignore')

# 데이터프레임용 라이브러리
import pandas as pd
# 위도경도 변환용 라이브러리
#!pip install geopy
from geopy.geocoders import GoogleV3

In [2]:
def check_element_exists(css_selector):
  try:
    test = driver.find_element(By.CSS_SELECTOR, css_selector)
    #print('false')
    return False
  except NoSuchElementException:
    #print('true')
    return True

In [ ]:
url = 'https://map.forest.go.kr/forest/'
# Google Geolocator 객체 생성
geolocator = GoogleV3(api_key="본인API")

options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome()
driver.get(url)

choso = []

btn = driver.find_element(By.CSS_SELECTOR, '#cookiecheck')
btn.click()
time.sleep(0.5)

search = driver.find_element(By.CSS_SELECTOR, '#poisearch-text').send_keys('산불감시초소')
time.sleep(0.5)
search_btn = driver.find_element(By.CSS_SELECTOR, '#poi-search-btn > input[type=image]')
search_btn.click()
time.sleep(0.5)

while True : 
  css_selector = "#result-table_wrapper > div:nth-child(3) > div:nth-child(2) > div > ul > li.next.disabled"
  is_exists = check_element_exists(css_selector)

  # 다음 버튼이 비활성화 될 경우 반복 중지
  if is_exists == False :
    break
  
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  infos = soup.select('#dorojusoResultTbody > tr')
  
  for info in infos :
    # 초소명
    name = info.select_one('#dorojusoResultTbody > tr > td:nth-child(1)').text
    address = info.select_one('#dorojusoResultTbody > tr > td:nth-child(2)').text
    try:
      # 주소를 위도와 경도로 변환
      location = geolocator.geocode(address, timeout=10)
      if location:
        print(f"위도: {location.latitude}, 경도: {location.longitude}")
        latitude = location.latitude
        longitude = location.longitude
      else:
        print("주소를 변환할 수 없습니다.")
    except Exception as e:
      print(f"오류 발생: {e}")
      latitude = 0
      longitude = 0
    time.sleep(0.3)
    choso.append([name, address, latitude, longitude])
  
  if is_exists == True :
    next_btn = driver.find_element(By.CSS_SELECTOR, '#result-table_wrapper > div:nth-child(3) > div:nth-child(2) > div > ul > li.next > a').click()
    time.sleep(0.5)
    
  # 다음 버튼이 비활성화 될 경우 반복 중지
  else : 
    break
driver.quit()

choso_df = pd.DataFrame(choso, columns = ['name', 'address', 'latitude', 'longitude'])
choso_df.info()
choso_df

In [5]:
choso_df = choso_df.drop(0).reset_index(drop=True)
choso_df.to_csv('초소이름.csv', encoding = 'utf-8', index = False)